# Mesh-generation with snappyHexMesh

We aim to compute the two-dimensional flow around an anatomically accurate cross-section of the flying-snake centered in the square domain [-15,15]x[-15,15] using the laminar incompressible solver in OpenFOAM: IcoFOAM.
To run a 2D simulation with OpenFOAM, we need to create a 3D mesh that contains only one cell in the third-direction and whose front and back faces are tagged with the special `empty` patch.
We choose snappyHexMesh, a 3D OpenFOAM mesh utility, to create the computational mesh.

We start by generating a background mesh of hexahedral cells in the domain [-15,15]x[-15,15]x[0,1] using another OpenFOAM mesh utility, blockMesh; at that point, we do not consider the bluff-body.
The characteristics of the mesh are provided by the file `blockMeshDict` located in the folder `constant/polyMesh` of the simulation directory.
We end up with a uniformly discretized mesh with a cell-width 0.25 (of the snake chord-length) in the x- and y-directions and only one division in the z-direction.
Starting from that base-mesh, snappyHexMesh generates a castellated mesh (where we choose the number of refinement levels in specific inner regions) that covers the entire domain, including the interior of the immersed body.
Then, the cells inside the body are removed while cells at the fluid/solid interface are snapped onto the bluff-body surface.
The surface of the body is defined in an .obj file located in the folder `constant/triSurface`.

As mentioned before, snappyHexMesh is a 3D mesh utility and the mesh-generation process can quickly become expensive (unnecessarily expensive when interested in a 2D simulation).
To overcome this problem, we incorporate extra steps in the process.
First, we use two-dimensional boxes (defined as .obj files located in the folder `constant/triSurface`) that snappyHexMesh considers as "surface features" and we provide the number of levels of refinement that are applied across each surface.
We use two boxes that covers the rectangular regions [-1,10]x[-2,2] and [-2,15]x[-4,4] and provide the number of levels of refinement to obtain a desired cell-width in each one. 
The first box is uniformly discretized with cell-width 0.004 (smallest characteristic-length used in Krishnan et al., 2014) while the second box has characteristic cell-width of 0.0016 to obtain a grid refined over the entire wake of the bluff-body.
In a second pass, we extrude the front face of mesh in the z-direction to a depth of one chord-length of the cross-section. 
By doing this, we greatly reduce the number of cells to snap onto the surface.

We end up with a high-quality mesh that contains about 3.4 million cells (mostly hexahedral cells and split hexahedra at the fluid/solid interface).
The bash script `runSnappyHexMesh.sh` (located in the simulation directory) contains all command-line instructions to generate the 2D mesh.